# Workshop 3

Starter code for workshop 3. You should have seen most of it before, but make sure you understand what it is doing!

In [48]:
# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [49]:
import pandas as pd

housing = pd.read_csv("workshop3.csv")
housing.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41,880,129.0,322,126,8.3252,452600
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,358500
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,352100
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,341300
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,342200
5,-122.25,37.85,52,919,213.0,413,193,4.0368,269700
6,-122.25,37.84,52,2535,489.0,1094,514,3.6591,299200
7,-122.25,37.84,52,3104,687.0,1157,647,3.1200,241400
8,-122.26,37.84,42,2555,665.0,1206,595,2.0804,226700
9,-122.25,37.84,52,3549,707.0,1551,714,3.6912,261100


Split the available data 80/20 for training and testing. Don't use the test data until the very end!

In [50]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

median_house_value is the value we want to predict, so separate it from the other features.

In [51]:
training_features = train_set.drop(["median_house_value"], axis=1) ##x
training_labels = train_set["median_house_value"].copy() ##y

training_features

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
17560,-121.80,37.32,14,4412,924.0,2698,891,4.7027
2045,-119.63,36.64,33,1036,181.0,620,174,3.4107
6383,-118.06,34.12,25,3891,848.0,1848,759,3.6639
4574,-118.31,34.07,28,2362,949.0,2759,894,2.2364
15118,-117.27,33.04,27,1839,392.0,1302,404,3.5500
...,...,...,...,...,...,...,...,...
11284,-117.97,33.72,24,2991,500.0,1437,453,5.4286
11964,-117.54,33.76,5,5846,1035.0,3258,1001,4.7965
5390,-118.42,34.01,42,1594,369.0,952,362,3.0990
860,-122.04,37.57,12,5719,1064.0,3436,1057,5.2879


Your code starts here...

In [52]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(training_features, training_labels)
prediction_lr = lr.predict(training_features)

In [53]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(training_labels, prediction_lr)
rmse = mean_squared_error(training_labels, prediction_lr, squared = False)

In [54]:
print(mse)
print(rmse)

4817977906.765843
69411.65541006671


In [8]:
training_labels.mean()

206644.40009788328

In [11]:
baseline = np.full(training_labels.shape, training_labels.mean())

In [12]:
rmse_bl = mean_squared_error(training_labels, baseline, squared = False)

In [13]:
rmse_bl

115051.63379493028

In [19]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors = 1)

knn.fit(training_features, training_labels)

knn_pred = knn.predict(training_features)

In [21]:
rmse_knn = mean_squared_error(training_labels, knn_pred, squared = False)
print(rmse_knn)

0.0


In [61]:
from sklearn.model_selection import cross_val_score

knn = KNeighborsRegressor(n_neighbors = 5)

scores_cv = cross_val_score(knn, training_features, training_labels, cv = 5, scoring = 'neg_root_mean_squared_error')

In [62]:
scores_cv

array([ -96858.31520441,  -99507.11514268,  -98768.88184361,
       -100897.33191843,  -98848.18874065])

In [26]:
-scores_cv.mean()

-98975.96656995492

In [31]:
from sklearn.model_selection import cross_validate

scores_cv2 = cross_validate(knn, training_features, training_labels, cv = 5 , scoring = 'neg_root_mean_squared_error')

scores_cv2

{'fit_time': array([0.03295588, 0.01601219, 0.01600075, 0.01600099, 0.01600099]),
 'score_time': array([0.0241437 , 0.02399707, 0.02403951, 0.02399731, 0.02399874]),
 'test_score': array([ -96858.31520441,  -99507.11514268,  -98768.88184361,
        -100897.33191843,  -98848.18874065])}

In [34]:
-scores_cv2['test_score'].mean()

98975.96656995492

In [35]:
knn = KNeighborsRegressor()

In [36]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [45]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_neighbors' :[3,5,9,11], 'weights':['uniform', 'distance']}

cif = GridSearchCV(knn, parameters, scoring = 'neg_root_mean_squared_error')

cif.fit(training_features, training_labels)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [3, 5, 9, 11],
                         'weights': ['uniform', 'distance']},
             scoring='neg_root_mean_squared_error')

In [46]:

cif.best_params_

{'n_neighbors': 11, 'weights': 'distance'}

In [47]:
cif.best_score_

-95640.04163755028